In [1]:
import tensorflow as tf
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [2]:
class ReinforcementAI_trader():
    def __init__(self, state_size, action_space = 3, model_name = 'RLtrader'):

    
        self.state_size = state_size
        self.action_space = action_space
        self.model_name = model_name
        self.memory = deque(maxlen = 2000) 
        self.inventory = []
    
        self.gamma = 0.95 #Bellman 
        self.epsilon = 1.0 
        self.epsilon_decay = 0.995 
        self.epsilon_final = 0.01 
    
        self.model = self.model_builder()
    
    def model_builder(self):
        
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(32, activation = 'relu', input_dim = self.state_size))
        model.add(tf.keras.layers.Dense(64, activation = 'relu'))
        model.add(tf.keras.layers.Dense(128, activation = 'relu'))
        model.add(tf.keras.layers.Dense(units = self.action_space , activation = 'linear'))
        model.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(lr = 0.01))
        
        return model
    
    def trade(self, state):
        if random.random() <= self.epsilon: 
            return random.randrange(self.action_space)
        
        actions = self.model.predict(state)
        print('actions: ', actions)
        print(actions[0])
        return np.argmax(actions[0]) 
    
    def batch_train(self, batch_size):
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)): 
            batch.append(self.memory[i])
            
        
        for state, action, reward, next_state, done in batch:
            reward = reward
            
            if not done: 
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0]) 
                
            target = self.model.predict(state)
            target[0][action] = reward
            print('target[0][action]: ', target[0][action])
            
            self.model.fit(state, target, epochs = 1, verbose = 0)
            
        
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay
            

In [3]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [4]:
def  stock_price_format(n):
    if n < 0: #si perdemos dinero lo visualizamos aqui, con un '-'
        return '- $ {0:2f}'.format(abs(n)) #{0:2f} de cero a dos cifras decimales
    else: #si ganamos dinero
        return '$ {0:2f}'.format(abs(n))

In [5]:
def data_loader(stock_name = 'TSLA'):
    dataset = data_reader.DataReader(stock_name, data_source = 'yahoo')
    
    
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    
    return close

---

In [6]:
def state_creator(data, timestep, window_size):
    
    starting_id = timestep - window_size + 1
    
    
    if starting_id >= 0:
        windowed_data = data[starting_id : timestep + 1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1]) 
        
    state = []
    for i in range(window_size -1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
        print('State:** ', state)
    
    return np.array([state])        

In [7]:
stock_name = 'TSLA'
data = data_loader(stock_name)

In [8]:
data.tail()

Date
2021-01-28    835.429993
2021-01-29    793.530029
2021-02-01    839.809998
2021-02-02    872.789978
2021-02-03    854.690002
Name: Close, dtype: float64

In [9]:
window_size = 10 
episodes = 10 

batch_size = 32
data_samples = len(data) - 1 

In [10]:
trader = ReinforcementAI_trader(window_size)

In [11]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


#### Bucle de entrenamiento

In [12]:
for episode in range(1, episodes + 1): 
    print('Epoch: {}/{}'.format(episode, episodes))
    
    state = state_creator(data, 0, window_size + 1) 
    
    
    total_profit = 0
    trader.inventory = []
    
    
    for t in tqdm(range(data_samples)): 
        action = trader.trade(state) 
        
        next_state = state_creator(data, t+1, window_size +1)
        
        
        reward = 0
        
        if action == 1:
            trader.inventory.append(data[t]) 
            print('AI trader ha comprado' , stock_price_format(data[t]))
        elif action == 2 and len(trader.inventory) > 0:
            buy_price = trader.inventory.pop(0)
            
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader vendió: ", stock_price_format(data[t]), " Beneficio: " + stock_price_format(data[t] - buy_price))
            
        if t == data_samples - 1: 
            done = True
        else:
            done = False
            
        
        trader.memory.append((state, action, reward, next_state, done))
        
        state = next_state
        
        if done:
            print("########################")
            print("BENEFICIO TOTAL: {}".format(total_profit))
            print("########################")
            
        #Si la longitud de la memoria es mayor al batch size, tiramos de memoria
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
        
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode)) #guardamos cada 10%

Epoch: 1/1000
State:**  [0.5]
State:**  [0.5, 0.5]
State:**  [0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


  0%|          | 0/1257 [00:00<?, ?it/s]

State:**  [0.5]
State:**  [0.5, 0.5]
State:**  [0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.051076635241176824]
State:**  [0.5]
State:**  [0.5, 0.5]
State:**  [0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.051076635241176824]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.051076635241176824, 0.5129970828949009]
State:**  [0.5]
State:**  [0.5, 0.5]
State:**  [0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5]
State:**  [0.5, 0.5, 0.5, 0.5, 0.5

  1%|          | 7/1257 [00:00<00:22, 56.60it/s]

State:**  [0.5]
State:**  [0.5, 0.5]
State:**  [0.5, 0.5, 0.051076635241176824]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009, 0.2857735476998378]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009, 0.2857735476998378, 0.7957597969140111]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009, 0.2857735476998378, 0.7957597969140111, 0.5284692544090536]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009, 0.2857735476998378, 0.7957597969140111, 0.5284692544090536, 0.6955085324138643]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009, 0.2857735476998378, 0.7957597969140111, 0.5284692544090536, 0.6955085324138643, 0.9371445364931144]
State:**  [0.5, 0.5, 0.051076635241176824, 0.5129970828949009, 0.2857735476998378, 0.7957597969140111, 0.5284692544090536, 0.6955085324138643, 0.9371445364931144, 0.4056446194504002]
AI trader ha comprado $ 33.736000
State:**  [0.5]
State:** 

...

 12%|█▏        | 152/1257 [06:38<57:19,  3.11s/it]

actions:  [[2.1819377 2.2352405 2.2984812]]
[2.1819377 2.2352405 2.2984812]
State:**  [0.5334496816542259]
State:**  [0.5334496816542259, 0.09552202554152811]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739, 0.7330205673023916]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015, 0.3593931271160259]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015, 0.3593931271160259, 0.6826547276835419]
State:**  [0.5334496816542259, 0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.444232285000

 12%|█▏        | 153/1257 [06:41<57:17,  3.11s/it]

actions:  [[1.6116384 1.654148  1.7019924]]
[1.6116384 1.654148  1.7019924]
State:**  [0.09552202554152811]
State:**  [0.09552202554152811, 0.3548014932408739]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015, 0.3593931271160259]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015, 0.3593931271160259, 0.6826547276835419]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.29525452425452015, 0.3593931271160259, 0.6826547276835419, 0.38936058465510254]
State:**  [0.09552202554152811, 0.3548014932408739, 0.7330205673023916, 0.4442322850002772, 0.295254524

KeyboardInterrupt: 